In [3]:
# Importing required libraries
import ffmpeg

In [4]:
# Step 1: Extract audio from video
def extract_audio(video_path, output_audio_path):
    ffmpeg.input(video_path).output(output_audio_path).run()

In [5]:
fileName = 'AI_Video_Tamil'
inputPath = './' + fileName + '.mp4'
outputPath = './' + fileName + '.mp3'

outputFileName = extract_audio(inputPath, outputPath)